In [1]:
import os, random, time
import xgboost
import datetime
import pygam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization
#from fbprophet import Prophet
from sklearn.linear_model import LinearRegression as LR
from sklearn.kernel_ridge import KernelRidge as KR
from sklearn.linear_model import Ridge, ElasticNet, BayesianRidge, Lars, Lasso
from sklearn.ensemble import RandomForestRegressor as RF

from sklearn.metrics import r2_score
from sklearn.gaussian_process import GaussianProcessRegressor as gpr
from sklearn.gaussian_process.kernels import WhiteKernel, ConstantKernel as C, RBF
import warnings

In [2]:
y_inven = pd.read_csv('./E61D_Inven.csv')
x_sales = pd.read_csv('./E61D_Sale.csv')
x_product = pd.read_csv('./E61D_Product.csv')
d_set = pd.merge(pd.merge(y_inven, x_sales, how='left', on='Week'), x_product, how='left', on='Week')
d_set = d_set.drop(columns=['5xxx', '3xxC', '3xxT'])
d_set[d_set < -1000] = 0
#d_set_log = d_set.copy()
#d_set_log[d_set < 0] = 0
#d_set.to_csv('data_0506.csv', index=False)

rm_week = [201952, 202001, 202052, 202053, 202101]
rm_index = [i for i,s in enumerate(d_set['Week']) if s not in rm_week]
d_set1 = d_set.loc[rm_index]
d_set1 = d_set1.reset_index(drop=True)
#print(d_set.shape)
#d_set1.to_csv('data_0506.csv', index=False)

In [30]:
def run_model_1(m_, x_dta, wt_mat, t_col_k, s_col_k, p_col_k, tr_s_day, tr_e_day, te_s_day=None, te_e_day=None):    
    tr_time = np.array([tr_s_day, tr_e_day])
    te_time = np.array([te_s_day, te_e_day])
    s_ind, e_ind = np.where(tr_time[0] == x_dta['Week'])[0][0], np.where(tr_time[1] == x_dta['Week'])[0][0]
    te_s_ind, te_e_ind = np.where(te_time[0] == x_dta['Week'])[0][0], np.where(te_time[1] == x_dta['Week'])[0][0]

    target_col = [s for s in x_dta.columns.values if 'Inven' == s]
    week_col = [s for s in x_dta.columns.values if 'Week' in s]

    prev_target_col = [s for s in x_dta.columns.values if 'Prev_Inven' in s]
    var_sale_col = [s for s in x_dta.columns.values if 'Var_Sale' in s]
    var_product_col = [s for s in x_dta.columns.values if 'Var_Product' in s]
    sale_col = [s for s in x_dta.columns.values if 'Sale' == s]
    product_col = [s for s in x_dta.columns.values if 'Product' == s]

    tmp_x_set1, tmp_x_set2, tmp_x_set3 = x_dta[prev_target_col], x_dta[var_sale_col], x_dta[var_product_col]    
    tmp_x_set1 = np.multiply(tmp_x_set1, np.tile([wt_mat[:tmp_x_set1.shape[1]]], tmp_x_set1.shape[0]).reshape(tmp_x_set1.shape[0], -1)).copy()
    tmp_x_set2 = np.multiply(tmp_x_set2, np.tile([wt_mat], tmp_x_set2.shape[0]).reshape(tmp_x_set2.shape[0], -1)).copy()
    tmp_x_set3 = np.multiply(tmp_x_set3, np.tile([wt_mat], tmp_x_set3.shape[0]).reshape(tmp_x_set3.shape[0], -1)).copy()

    tmp_x_set1_re = tmp_x_set1.iloc[:,:t_col_k]
    tmp_x_set2_re = tmp_x_set2.iloc[:,:s_col_k]
    tmp_x_set3_re = tmp_x_set3.iloc[:,:p_col_k]

    x_dta_re = pd.concat([x_dta[list(week_col+target_col+sale_col+product_col)], tmp_x_set1_re, tmp_x_set2_re, tmp_x_set3_re], axis=1)

    train_set = x_dta_re.iloc[s_ind:e_ind, :]
    test_set = x_dta_re.iloc[te_s_ind:(te_e_ind+1), :]#.copy()

    tr_x_set = train_set[list(prev_target_col[:t_col_k]+sale_col+var_sale_col[:s_col_k]+product_col+var_product_col[:p_col_k])]
    tr_y_set = np.array(train_set[target_col]).reshape(-1,1)
    te_x_set = test_set[list(prev_target_col[:t_col_k]+sale_col+var_sale_col[:s_col_k]+product_col+var_product_col[:p_col_k])]
    te_y_set = np.array(test_set[target_col]).reshape(-1,1)


    m_.fit(tr_x_set, tr_y_set)
    hat_prev_ = m_.predict(tr_x_set)
    if len(np.shape(hat_prev_)) == 1:
        hat_prev_ = hat_prev_.reshape(-1,1)

    real_ = np.mean(np.ones_like(tr_y_set) - np.clip(np.abs(tr_y_set - hat_prev_) / tr_y_set , 0, 1)) * 100   

    hat_ = []
    prev_x = list(map(lambda j : x_dta_re['Inven'].iloc[te_s_ind-j], range(1, t_col_k+1)))
    for i in range(0, te_x_set.shape[0]):    
        var_x = te_x_set[list(sale_col+var_sale_col[:s_col_k]+product_col+var_product_col[:p_col_k])].iloc[i,:]
        te_x_ = np.array(prev_x+list(var_x)).reshape(1,-1)    
        tmp_y_hat_ = m_.predict(te_x_)
        if len(np.shape(tmp_y_hat_)) > 1:
            tmp_y_hat_ = tmp_y_hat_.reshape(-1, )            
        prev_x = (list(tmp_y_hat_) + prev_x)[:t_col_k]
#            if len(np.shape(tmp_y_hat_)) == 1:
#                tmp_y_hat_ = tmp_y_hat_.reshape(-1,1)        
        hat_.append(tmp_y_hat_)
    Y_hat_ = np.concatenate((hat_prev_, hat_)) #hat_))#    
    Y_     = np.concatenate((tr_y_set, te_y_set))
    fcst_ = np.mean(np.ones_like(te_y_set) - np.clip(np.abs(te_y_set - hat_[0]) / te_y_set , 0, 1)) * 100    
    return real_, fcst_, Y_, Y_hat_, hat_prev_, hat_

In [32]:
def run_model_2(m_, x_dta, target_key, wt_mat, tr_s_day, tr_e_day, te_s_day=None, te_e_day=None, col_k=None, col_k1=None):    
    
    tr_time = np.array([tr_s_day, tr_e_day])
    te_time = np.array([te_s_day, te_e_day])
    s_ind, e_ind = np.where(tr_time[0] == x_dta['Week'])[0][0], np.where(tr_time[1] == x_dta['Week'])[0][0]
    te_s_ind, te_e_ind = np.where(te_time[0] == x_dta['Week'])[0][0], np.where(te_time[1] == x_dta['Week'])[0][0]

    target_col = [s for s in x_dta.columns.values if target_key == s]    
    tmp_x_col = [s for s in x_dta.columns.values if 'Prev_'+target_key in s]
    tmp_x_col1 = [s for s in x_dta.columns.values if 'Var_'+target_key in s]
    week_col = [s for s in x_dta.columns.values if 'Week' in s]

    tmp_x_set1, tmp_x_set2 = x_dta[tmp_x_col], x_dta[tmp_x_col1]
    tmp_x_set1 = np.multiply(tmp_x_set1, np.tile([wt_mat], tmp_x_set1.shape[0]).reshape(tmp_x_set1.shape[0], -1)).copy()
    tmp_x_set2 = np.multiply(tmp_x_set2, np.tile([wt_mat], tmp_x_set2.shape[0]).reshape(tmp_x_set2.shape[0], -1)).copy()

    if target_key == 'Product':
        tmp_x_set1_re = tmp_x_set1.iloc[:,:col_k]
        tmp_x_set2_re = tmp_x_set2.iloc[:,:col_k1]
        x_dta_re = pd.concat([x_dta[list(week_col+target_col)], tmp_x_set1_re, tmp_x_set2_re], axis=1)
        train_col = list(tmp_x_col[:col_k]+tmp_x_col1[:col_k1])    
    elif target_key == 'Sale':
        tmp_x_set2_re = tmp_x_set2.iloc[:,:col_k1]
        x_dta_re = pd.concat([x_dta[list(week_col+target_col)], tmp_x_set2_re], axis=1)
        train_col = list(tmp_x_col1[:col_k1])


    train_set = x_dta_re.iloc[s_ind:e_ind, :]
    test_set = x_dta_re.iloc[te_s_ind:(te_e_ind+1), :]#.copy()

    tr_x_set = train_set[train_col]
    tr_y_set = np.array(train_set[target_col]).reshape(-1,1)
    te_x_set = test_set[train_col]
    te_y_set = np.array(test_set[target_col]).reshape(-1,1)

    m_.fit(tr_x_set, tr_y_set)
    hat_prev_ = m_.predict(tr_x_set)
    if len(np.shape(hat_prev_)) == 1:
        hat_prev_ = hat_prev_.reshape(-1,1)

    real_ = np.mean(np.ones_like(tr_y_set) - np.clip(np.abs(tr_y_set - hat_prev_) / tr_y_set , 0, 1)) * 100    
    #if len(tr_y_set) > 0 :
    #    tmp_y_id_ = np.where(tr_y_set!=0)[0]
    #    real_ = np.mean(1- np.abs(tr_y_set[tmp_y_id_] - hat_prev_[tmp_y_id_]) / np.abs(tr_y_set[tmp_y_id_])) * 100
    #else:
    #    real_ = np.mean(1- np.abs(tr_y_set - hat_prev_) / np.abs(tr_y_set)) * 100     


    hat_ = []
    if target_key == 'Product':
        prev_x = list(map(lambda j : x_dta_re[target_key].iloc[te_s_ind-j], range(1, col_k+1)))
        for i in range(0, te_x_set.shape[0]):    
            var_x = te_x_set[tmp_x_col1[:col_k1]].iloc[i, :]        
            te_x_ = np.array(prev_x+list(var_x)).reshape(1,-1)    
            #print(te_x_.shape)
            tmp_y_hat_ = m_.predict(te_x_)
            if len(np.shape(tmp_y_hat_)) > 1:
                tmp_y_hat_ = tmp_y_hat_.reshape(-1, )            
            prev_x = (list(tmp_y_hat_) + prev_x)[:col_k]
#            if len(np.shape(tmp_y_hat_)) == 1:
#                tmp_y_hat_ = tmp_y_hat_.reshape(-1,1)        
            hat_.append(tmp_y_hat_)
        Y_hat_ = np.concatenate((hat_prev_, hat_)) #hat_))#
            
    elif target_key == 'Sale':
        tmp_y_hat_ = m_.predict(te_x_set)
        if len(np.shape(tmp_y_hat_)) == 1:
            tmp_y_hat_ = tmp_y_hat_.reshape(-1, 1)             
        hat_.append(tmp_y_hat_) 
        Y_hat_ = np.concatenate((hat_prev_, hat_[0])) #hat_))#
    
    Y_     = np.concatenate((tr_y_set, te_y_set))
    fcst_ = np.mean(np.ones_like(te_y_set) - np.clip(np.abs(te_y_set - hat_[0]) / te_y_set , 0, 1)) * 100    
    return real_, fcst_, Y_, Y_hat_, hat_prev_, hat_

In [36]:
warnings.filterwarnings(action='ignore', category=UserWarning)
param_bound = {'alpha' : (0.85,0.99) , 'm_n_estimator' : (10, 100), 'm_lr' : (0.01, 0.99), 'm_subsample' : (0.5, 0.9),
               'm_max_depth' : (3,9), 't_col_k' : (1,8.4), 's_col_k' : (1,12.4), 'p_col_k' : (1,12.4)}
def h_opt(alpha, m_n_estimator, m_lr, m_subsample, m_max_depth, t_col_k, s_col_k, p_col_k):     
    weight_mat = list(map(lambda x : round(alpha,3)**x if x > 0 else 1, range(0,12)))    
    #model_=xgboost.XGBRegressor(n_estimators=round(m_n_estimator), learning_rate=round(m_lr,3), 
                                subsample=round(m_subsample,3), max_depth=round(m_max_depth), tree_method='gpu_hist', gpu_id=0)
    #model_= Ridge()#LR(m_lr) #    
    model_ = KR(m_lr)
    acc_, acc_hat_, _, _, _, _  = run_model_1(model_, d_set1, weight_mat, 
                                              round(t_col_k), round(s_col_k), round(p_col_k), 202008, 202041, 202041, 202102)# # prev :/ var : 
    return acc_hat_    

sale_optimizer = BayesianOptimization(f=h_opt, pbounds=param_bound, verbose=2, random_state=1)
sale_optimizer.maximize(init_points=30, n_iter=120)
####################################################################################################################################
#####    |  target   |   alpha   |   m_lr    | m_max_... | m_n_es... | m_subs... |  p_col_k  |  s_col_k  |  t_col_k  |
#####    |  86.88    |  0.9218   |  0.5155   |  3.912    |  52.12    |  0.6188   |  9.2      |  6.628    |  5.863    |   => XGBoost
#####    |  90.48    |  0.85     |  0.01     |  5.05     |  68.06    |  0.9      |  1.0      |  1.0      |  6.371    |   => KR
#####    |  93.21    |  0.99     |  0.99     |  6.08     |  65.54    |  0.5      |  1.0      |  1.0      |  6.13     |   => LR
#####    |  93.21    |  0.85     |  0.99     |  5.482    |  88.84    |  0.9      |  1.0      |  1.0      |  5.534    |   => Ridge


|   iter    |  target   |   alpha   |   m_lr    | m_max_... | m_n_es... | m_subs... |  p_col_k  |  s_col_k  |  t_col_k  |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  60.18    |  0.9084   |  0.7159   |  3.001    |  37.21    |  0.5587   |  2.053    |  3.123    |  3.557    |
|  2        |  59.21    |  0.9055   |  0.538    |  5.515    |  71.67    |  0.5818   |  11.01    |  1.312    |  5.961    |
|  3        |  57.18    |  0.9084   |  0.5575   |  3.842    |  27.83    |  0.8203   |  12.04    |  4.573    |  6.123    |
|  4        |  57.58    |  0.9727   |  0.8867   |  3.51     |  13.51    |  0.5679   |  11.01    |  2.121    |  4.116    |
|  5        |  58.97    |  0.9841   |  0.5325   |  7.151    |  38.4     |  0.7746   |  10.51    |  1.208    |  6.551    |
|  6        |  33.95    |  0.9884   |  0.7432   |  4.683    |  81.04    |  0.5413   |  6.106    |  11.36    |  3.173    |
|  7        |  57.8     

In [22]:
warnings.filterwarnings(action='ignore', category=UserWarning)
param_bound = {'alpha' : (0.85,0.99) , 'm_n_estimator' : (10, 100), 'm_lr' : (0.01, 0.3), 'm_subsample' : (0.4, 0.9), 'm_max_depth' : (3,9), 'col_k' : (1,13), 'col_k1' : (1,13)}
def h_opt(alpha, m_n_estimator, m_lr, m_subsample, m_max_depth, col_k, col_k1):
    weight_mat = list(map(lambda x : round(alpha,3)**x if x > 0 else 1, range(0,12)))    #
    #model_=xgboost.XGBRegressor(n_estimators=round(m_n_estimator), learning_rate=round(m_lr,3), subsample=round(m_subsample,3), max_depth=round(m_max_depth), tree_method='gpu_hist', gpu_id=0)
    model_ = KR(m_lr)
    #acc_, acc_hat_, _, _, _, _ = run_model_(model_, d_set, 'Product', round(col_k), 201903, 202043, 202044, 202053)
    acc_, acc_hat_, _, _, _, _ = run_model_2(model_, d_set1, 'Sale', weight_mat, 201945, 202102, 202102, 202112, None, round(col_k1))  # prev / var : 
    return acc_hat_    

product_optimizer = BayesianOptimization(f=h_opt, pbounds=param_bound, verbose=2, random_state=1)
product_optimizer.maximize(init_points=10, n_iter=100)
#####################################################################################################################
#####    |  target   |   alpha   |   col_k   |  col_k1   |   m_lr    | m_max_... | m_n_es... | m_subs... |
#####    |  76.57    |  0.907    |  1.101    |  7.027    |  0.4475   |  7.76     |  74.95    |  0.4331   |   => XGBoost
#####    |  73.9     |  0.85     |  3.69     |  8.361    |  0.01     |  5.146    |  27.73    |  0.9      |

|   iter    |  target   |   alpha   |   col_k   |  col_k1   |   m_lr    | m_max_... | m_n_es... | m_subs... |
-------------------------------------------------------------------------------------------------------------
|  1        |  72.39    |  0.9084   |  9.644    |  1.001    |  0.09768  |  3.881    |  18.31    |  0.4931   |
|  2        |  73.23    |  0.8984   |  5.761    |  7.466    |  0.1316   |  7.111    |  28.4     |  0.8391   |
|  3        |  72.44    |  0.8538   |  9.046    |  6.008    |  0.172    |  3.842    |  27.83    |  0.8004   |
|  4        |  69.08    |  0.9856   |  4.761    |  9.308    |  0.2642   |  8.368    |  17.65    |  0.4195   |
|  5        |  72.33    |  0.8738   |  11.54    |  2.18     |  0.1321   |  8.747    |  57.98    |  0.7459   |
|  6        |  67.38    |  0.8942   |  9.238    |  11.02    |  0.0153   |  7.501    |  99.0     |  0.7741   |
|  7        |  72.32    |  0.8893   |  10.47    |  2.239    |  0.1399   |  8.452    |  36.43    |  0.5439   |
|  8      

/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/envs/rapids/lib/python3.7/site-packages/ipyk

|  11       |  73.65    |  0.9518   |  3.341    |  7.659    |  0.2618   |  3.282    |  44.69    |  0.8599   |


/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide


|  12       |  72.48    |  0.9076   |  6.236    |  5.622    |  0.1738   |  6.105    |  46.16    |  0.6559   |


...

/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide


|  108      |  73.9     |  0.85     |  10.02    |  8.214    |  0.01     |  6.265    |  38.83    |  0.9      |


/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide


|  109      |  73.9     |  0.8501   |  9.195    |  7.528    |  0.01     |  5.931    |  36.84    |  0.9      |
|  110      |  73.57    |  0.9178   |  7.713    |  6.69     |  0.01     |  5.033    |  35.99    |  0.4      |


/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide


In [33]:
warnings.filterwarnings(action='ignore', category=UserWarning)
param_bound = {'alpha' : (0.85,0.99) , 'm_n_estimator' : (10, 100), 'm_lr' : (0.01, 0.3), 'm_subsample' : (0.4, 0.9), 'm_max_depth' : (3,9), 'col_k' : (1,13), 'col_k1' : (1,13)}
def h_opt(alpha, m_n_estimator, m_lr, m_subsample, m_max_depth, col_k, col_k1):
    weight_mat = list(map(lambda x : round(alpha,3)**x if x > 0 else 1, range(0,12)))    #
    #model_=xgboost.XGBRegressor(n_estimators=round(m_n_estimator), learning_rate=round(m_lr,3), subsample=round(m_subsample,3), max_depth=round(m_max_depth), tree_method='gpu_hist', gpu_id=0)
    model_ = KR(m_lr)
    #acc_, acc_hat_, _, _, _, _ = run_model_(model_, d_set, 'Product', round(col_k), 201903, 202043, 202044, 202053)
    acc_, acc_hat_, _, _, _, _ = run_model_2(model_, d_set1, 'Product', weight_mat, 201945, 202102, 202102, 202112,  round(col_k), round(col_k1))  # prev/ var : 
    return acc_hat_    

product_optimizer = BayesianOptimization(f=h_opt, pbounds=param_bound, verbose=2, random_state=1)
product_optimizer.maximize(init_points=10, n_iter=100)
#####################################################################################################################
#####    |  target   |   alpha   |   col_k   |  col_k1   |   m_lr    | m_max_... | m_n_es... | m_subs... |
#####    |  89.51    |  0.9372   |  2.36     |  1.64     |  0.486    |  6.004    |  18.09    |  0.5309   |
#####    |  79.49    |  0.9851   |  1.796    |  1.627    |  0.1873   |  6.474    |  87.47    |  0.6013   |    => KR

|   iter    |  target   |   alpha   |   col_k   |  col_k1   |   m_lr    | m_max_... | m_n_es... | m_subs... |
-------------------------------------------------------------------------------------------------------------
|  1        |  62.17    |  0.9084   |  9.644    |  1.001    |  0.09768  |  3.881    |  18.31    |  0.4931   |
|  2        |  60.94    |  0.8984   |  5.761    |  7.466    |  0.1316   |  7.111    |  28.4     |  0.8391   |
|  3        |  22.65    |  0.8538   |  9.046    |  6.008    |  0.172    |  3.842    |  27.83    |  0.8004   |
|  4        |  76.24    |  0.9856   |  4.761    |  9.308    |  0.2642   |  8.368    |  17.65    |  0.4195   |
|  5        |  43.17    |  0.8738   |  11.54    |  2.18     |  0.1321   |  8.747    |  57.98    |  0.7459   |
|  6        |  42.43    |  0.8942   |  9.238    |  11.02    |  0.0153   |  7.501    |  99.0     |  0.7741   |
|  7        |  54.36    |  0.8893   |  10.47    |  2.239    |  0.1399   |  8.452    |  36.43    |  0.5439   |
|  8      

/opt/conda/envs/rapids/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:188: LinAlgWarning: Ill-conditioned matrix (rcond=5.92949e-17): result may not be accurate.
  overwrite_a=False)


|  9        |  74.68    |  0.9304   |  2.761    |  8.072    |  0.2129   |  3.614    |  47.27    |  0.7472   |
|  10       |  74.83    |  0.908    |  1.599    |  7.431    |  0.2025   |  6.089    |  95.01    |  0.6933   |
|  11       |  57.8     |  0.875    |  5.054    |  9.044    |  0.06062  |  8.362    |  18.21    |  0.7722   |
|  12       |  75.25    |  0.9571   |  2.314    |  7.264    |  0.2328   |  7.123    |  21.97    |  0.501    |
|  13       |  79.49    |  0.9851   |  1.796    |  1.627    |  0.1873   |  6.474    |  87.47    |  0.6013   |
|  14       |  52.58    |  0.8952   |  9.553    |  6.759    |  0.01777  |  7.095    |  80.17    |  0.887    |
|  15       |  61.24    |  0.9118   |  7.108    |  8.006    |  0.2534   |  3.016    |  83.62    |  0.4556   |


/opt/conda/envs/rapids/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:188: LinAlgWarning: Ill-conditioned matrix (rcond=6.68563e-17): result may not be accurate.
  overwrite_a=False)


|  16       |  71.39    |  0.9503   |  8.819    |  2.255    |  0.03525  |  6.931    |  84.83    |  0.5009   |
|  17       |  76.86    |  0.9795   |  11.06    |  1.091    |  0.2693   |  3.032    |  55.79    |  0.6222   |
|  18       |  79.07    |  0.9875   |  3.377    |  12.15    |  0.1304   |  5.427    |  11.45    |  0.7903   |
|  19       |  62.26    |  0.938    |  10.94    |  8.476    |  0.1057   |  4.765    |  18.19    |  0.4068   |
|  20       |  75.42    |  0.9151   |  3.828    |  3.304    |  0.03807  |  8.566    |  48.62    |  0.6982   |
|  21       |  74.72    |  0.9656   |  10.33    |  9.608    |  0.2279   |  8.152    |  10.61    |  0.7536   |
|  22       |  70.87    |  0.9659   |  8.097    |  8.364    |  0.1693   |  6.588    |  67.46    |  0.5727   |


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 18 while Y.shape[1] == 17